In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 20429770
paper_name = 'mir_rashed_smith_2010' 

In [49]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [50]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [24]:
original_data = pd.read_excel('raw_data/Ech sup Table work 5.xls', sheet_name='Sheet1', skiprows=5)

In [25]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 230 x 47


In [26]:
original_data.head()

,Gene ID,% reduction,Unnamed: 2,Gene ID.1,% reduction.1,Unnamed: 5,Gene ID.2,% reduction.2,Unnamed: 8,Gene ID.3,...,% reduction.12,Unnamed: 38,Gene ID.13,% reduction.13,Unnamed: 41,Gene ID.14,% reduction.14,Unnamed: 44,Gene ID.15,% reduction.15
0,YAL008W,61.631,NaN,YJR031C,110.070,NaN,YMR207C,144.79,NaN,YDR501W,...,179.47,NaN,YHR132C,93.651,NaN,YGR207C,100.060,NaN,YBL031W,130.700
1,YHR003C,57.622,NaN,YER155C,99.667,NaN,YLR366W,140.32,NaN,YHR105W,...,176.27,NaN,YER086W,82.973,NaN,YHL010C,89.252,NaN,YKL073W,108.050
2,YGL054C,54.341,NaN,YJR133W,82.232,NaN,YMR014W,140.32,NaN,YIL053W,...,155.89,NaN,YER044C,78.295,NaN,YDR157W,88.511,NaN,YBR224W,107.260
3,YDL192W,52.177,NaN,YJL141C,79.846,NaN,YLR365W,139.85,NaN,YNL046W,...,154.82,NaN,YGL084C,72.632,NaN,YGR202C,85.546,NaN,YDL222C,97.573
4,YBL065W,50.730,NaN,YJL151C,79.462,NaN,YMR226C,139.15,NaN,YMR231W,...,148.77,NaN,YDR245W,70.064,NaN,YHR124W,83.393,NaN,YDL237W,95.739


In [27]:
original_data.columns.values

array(['Gene ID', '% reduction', 'Unnamed: 2', 'Gene ID.1',
       '% reduction.1', 'Unnamed: 5', 'Gene ID.2', '% reduction.2',
       'Unnamed: 8', 'Gene ID.3', '% reduction.3', 'Unnamed: 11',
       'Gene ID.4', '% reduction.4', 'Unnamed: 14', 'Gene ID.5',
       '% reduction.5', 'Unnamed: 17', 'Gene ID.6', '% reduction.6',
       'Unnamed: 20', 'Gene ID.7', '% reduction.7', 'Unnamed: 23',
       'Gene ID.8', '% reduction.8', 'Unnamed: 26', 'Gene ID.9',
       '% reduction.9', 'Unnamed: 29', 'Gene ID.10', '% reduction.10',
       'Unnamed: 32', 'Gene ID.11', '% reduction.11', 'Unnamed: 35',
       'Gene ID.12', '% reduction.12', 'Unnamed: 38', 'Gene ID.13',
       '% reduction.13', 'Unnamed: 41', 'Gene ID.14', '% reduction.14',
       'Unnamed: 44', 'Gene ID.15', '% reduction.15'], dtype=object)

In [29]:
cols_orfs = [ic for ic, c in enumerate(original_data.columns.values) if 'Gene ID' in c]
cols_orfs

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45]

In [30]:
original_data_list = []
for c in cols_orfs:
    t = original_data.iloc[:,[c,c+1]].copy()
    t.columns = [0,1]
    t['orf'] = t[0].astype(str)
    t['orf'] = clean_orf(t['orf'])
    t['orf'] = translate_sc(t['orf'], to='orf')
    tx = looks_like_orf(t['orf'])
    print(t.loc[~tx,])
    t = t.loc[tx,:]
    t['data'] = -t[1].astype(float)
    t.set_index('orf', inplace=True)
    t = t[['data']].copy()
    t = t.groupby(t.index).mean()
    
    original_data_list.append(t)

               0   1  orf
index_input              
226          NaN NaN  NAN
227          NaN NaN  NAN
228          NaN NaN  NAN
229          NaN NaN  NAN
               0   1  orf
index_input              
220          NaN NaN  NAN
221          NaN NaN  NAN
222          NaN NaN  NAN
223          NaN NaN  NAN
224          NaN NaN  NAN
225          NaN NaN  NAN
226          NaN NaN  NAN
227          NaN NaN  NAN
228          NaN NaN  NAN
229          NaN NaN  NAN
               0   1  orf
index_input              
228          NaN NaN  NAN
229          NaN NaN  NAN
               0   1  orf
index_input              
229          NaN NaN  NAN
               0   1  orf
index_input              
200          NaN NaN  NAN
201          NaN NaN  NAN
202          NaN NaN  NAN
203          NaN NaN  NAN
204          NaN NaN  NAN
205          NaN NaN  NAN
206          NaN NaN  NAN
207          NaN NaN  NAN
208          NaN NaN  NAN
209          NaN NaN  NAN
210          NaN NaN  NAN
211         

In [31]:
original_data = pd.concat(original_data_list, axis=1)

In [32]:
original_data.index.name='orf'
original_data[original_data.isnull()] = 0

In [45]:
col_names = ['SGEPH.Dark','SGEPH.UV','SGEPR.Dark','SGEPR.UV','SG1.Dark','SG1.UV',
             'SG7a.Dark','SG7a.UV','SGEPF.Dark','SGEPF.UV','SGEPLS.Dark',
             'SGEPLS.UV','SGEARa.Dark','SGEARa.UV','SGEARb.Dark','SGEARb.UV']

In [47]:
original_data.columns = col_names

In [48]:
original_data.head()

,SGEPH.Dark,SGEPH.UV,SGEPR.Dark,SGEPR.UV,SG1.Dark,SG1.UV,SG7a.Dark,SG7a.UV,SGEPF.Dark,SGEPF.UV,SGEPLS.Dark,SGEPLS.UV,SGEARa.Dark,SGEARa.UV,SGEARb.Dark,SGEARb.UV
orf,,,,,,,,,,,,,,,,
YAL004W,-24.418,0.000,0.0,0.0,0.000,-105.830,0.000,0.0,-59.33,-65.554,0.000,0.000,0.0,0.0,0.0,0.0
YAL008W,-61.631,0.000,0.0,0.0,0.000,0.000,-38.155,0.0,0.00,-53.989,-45.218,0.000,0.0,0.0,0.0,0.0
YAL027W,-26.858,-32.853,0.0,0.0,0.000,0.000,0.000,0.0,0.00,-60.884,0.000,0.000,0.0,0.0,0.0,0.0
YAL030W,-22.687,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.00,-59.395,0.000,0.000,0.0,0.0,0.0,0.0
YAL056C-A,-23.384,0.000,0.0,0.0,-40.014,-46.716,-85.502,0.0,0.00,0.000,-78.771,-43.633,0.0,0.0,0.0,0.0


# Prepare the final dataset

In [51]:
data = original_data.copy()

In [53]:
dataset_names = dict()
dataset_names['SG1.Dark'] = 16457
dataset_names['SG1.UV'] = 16575
dataset_names['SGEPR.Dark'] = 16576
dataset_names['SGEPR.UV'] = 16577
dataset_names['SGEPF.Dark'] = 16578
dataset_names['SGEPF.UV'] = 16579
dataset_names['SGEPH.Dark'] = 16580
dataset_names['SGEPH.UV'] = 16581
dataset_names['SGEPLS.Dark'] = 16582
dataset_names['SGEPLS.UV'] = 16583
dataset_names['SG7a.Dark'] = 16584
dataset_names['SG7a.UV'] = 16585
dataset_names['SGEARa.Dark'] = 16586
dataset_names['SGEARa.UV'] = 16587
dataset_names['SGEARb.Dark'] = 16588
dataset_names['SGEARb.UV'] = 16589

In [54]:
dataset_ids = [dataset_names[c] for c in data.columns.values]
datasets = datasets.reindex(index=dataset_ids)

In [55]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [56]:
data.head()

dataset_id,16580,16581,16576,16577,16457,16575,16584,16585,16578,16579,16582,16583,16586,16587,16588,16589
data_type,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value
orf,,,,,,,,,,,,,,,,
YAL004W,-24.418,0.000,0.0,0.0,0.000,-105.830,0.000,0.0,-59.33,-65.554,0.000,0.000,0.0,0.0,0.0,0.0
YAL008W,-61.631,0.000,0.0,0.0,0.000,0.000,-38.155,0.0,0.00,-53.989,-45.218,0.000,0.0,0.0,0.0,0.0
YAL027W,-26.858,-32.853,0.0,0.0,0.000,0.000,0.000,0.0,0.00,-60.884,0.000,0.000,0.0,0.0,0.0,0.0
YAL030W,-22.687,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.00,-59.395,0.000,0.000,0.0,0.0,0.0,0.0
YAL056C-A,-23.384,0.000,0.0,0.0,-40.014,-46.716,-85.502,0.0,0.00,0.000,-78.771,-43.633,0.0,0.0,0.0,0.0


## Subset to the genes currently in SGD

In [57]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 13


In [58]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16580,16581,16576,16577,16457,16575,16584,16585,16578,16579,16582,16583,16586,16587,16588,16589
,data_type,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value
gene_id,orf,,,,,,,,,,,,,,,,
1863,YAL004W,-24.418,0.000,0.0,0.0,0.000,-105.830,0.000,0.0,-59.33,-65.554,0.000,0.000,0.0,0.0,0.0,0.0
6,YAL008W,-61.631,0.000,0.0,0.0,0.000,0.000,-38.155,0.0,0.00,-53.989,-45.218,0.000,0.0,0.0,0.0,0.0
25,YAL027W,-26.858,-32.853,0.0,0.0,0.000,0.000,0.000,0.0,0.00,-60.884,0.000,0.000,0.0,0.0,0.0,0.0
28,YAL030W,-22.687,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.00,-59.395,0.000,0.000,0.0,0.0,0.0,0.0
1866,YAL056C-A,-23.384,0.000,0.0,0.0,-40.014,-46.716,-85.502,0.0,0.00,0.000,-78.771,-43.633,0.0,0.0,0.0,0.0


# Normalize

In [59]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [60]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [61]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,16580,16581,16576,16577,16457,16575,16584,16585,16578,16579,...,16584,16585,16578,16579,16582,16583,16586,16587,16588,16589
,data_type,value,value,value,value,value,value,value,value,value,value,...,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
1863,YAL004W,-24.418,0.000,0.0,0.0,0.000,-105.830,0.000,0.0,-59.33,-65.554,...,0.000000,0.0,-4.056515,-4.824526,0.000000,0.000000,0.0,0.0,0.0,0.0
6,YAL008W,-61.631,0.000,0.0,0.0,0.000,0.000,-38.155,0.0,0.00,-53.989,...,-2.862525,0.0,0.000000,-3.973386,-3.622478,0.000000,0.0,0.0,0.0,0.0
25,YAL027W,-26.858,-32.853,0.0,0.0,0.000,0.000,0.000,0.0,0.00,-60.884,...,0.000000,0.0,0.000000,-4.480832,0.000000,0.000000,0.0,0.0,0.0,0.0
28,YAL030W,-22.687,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.00,-59.395,...,0.000000,0.0,0.000000,-4.371247,0.000000,0.000000,0.0,0.0,0.0,0.0
1866,YAL056C-A,-23.384,0.000,0.0,0.0,-40.014,-46.716,-85.502,0.0,0.00,0.000,...,-6.414666,0.0,0.000000,0.000000,-6.310456,-4.481087,0.0,0.0,0.0,0.0


# Print out

In [62]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [63]:
from IO.save_data_to_db3 import *

In [64]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/16 [00:00<?, ?it/s]

Deleting all datasets for PMID 20429770...
Inserting the new data...


100%|██████████| 16/16 [00:55<00:00,  3.49s/it]

Updating the data_modified_on field...
